In [1]:
import numpy as np
from scipy.integrate import quad
import cmath
import numpy as np
from scipy.integrate import quad
from scipy.stats import norm

In [2]:
print("\n" + "="*60)
print("Fourier transformations")
print("="*60)

print("\n" + "="*60)
print("Inverse Transform of a Gaussian")
print("="*60)

# --- Example 2.1: Inverse Transform of a Gaussian ---

# The text calculates f(x) from f_hat(p)
# f(x) = integral[ dp/(2*pi) * exp(ipx) * f_hat(p) ]
# Where f_hat(p) = exp(-sigma^2 * p^2 / 2)

def f_hat_p(p, sigma):
    return np.exp(- (sigma**2 * p**2) / 2)

def integrand_ex2_1(p, x, sigma):
    # exp(ipx) * f_hat(p)
    return cmath.exp(1j * p * x) * f_hat_p(p, sigma)

def calculate_f_x(x, sigma):
    # We integrate the real and imaginary parts separately
    real_part, _ = quad(lambda p: integrand_ex2_1(p, x, sigma).real, -np.inf, np.inf)
    imag_part, _ = quad(lambda p: integrand_ex2_1(p, x, sigma).imag, -np.inf, np.inf)
    
    # We divide by 2*pi
    return (real_part + 1j * imag_part) / (2 * np.pi)

# Analytical result from the document
def analytical_f_x(x, sigma):
    return (1 / np.sqrt(2 * np.pi * sigma**2)) * np.exp(- (x**2) / (2 * sigma**2))

print("--- Example 2.1: Fourier Transform (Inverse) ---")
x_val = 1.0
sigma_val = 2.0
numeric_result = calculate_f_x(x_val, sigma_val)
analytical_result = analytical_f_x(x_val, sigma_val)

print(f"f(x={x_val}, σ={sigma_val}) (Numeric):    {numeric_result.real:.6f}")
print(f"f(x={x_val}, σ={sigma_val}) (Analytical): {analytical_result:.6f}")


print("\n" + "="*60)
print("Transform of f(x) = exp(-m|x|)")
print("="*60)

# --- Example 2.2: Transform of f(x) = exp(-m|x|) ---

# The text calculates f_hat(p) from f(x)
# f_hat(p) = integral[ dx * exp(-ipx) * f(x) ]

def f_x_ex2_2(x, m):
    return np.exp(-m * np.abs(x))

def integrand_ex2_2(x, p, m):
    # exp(-ipx) * f(x)
    return cmath.exp(-1j * p * x) * f_x_ex2_2(x, m)

def calculate_f_hat_p(p, m):
    real_part, _ = quad(lambda x: integrand_ex2_2(x, p, m).real, -np.inf, np.inf)
    imag_part, _ = quad(lambda x: integrand_ex2_2(x, p, m).imag, -np.inf, np.inf)
    
    return real_part + 1j * imag_part

# Analytical result from the document (using 'p' instead of 'q')
def analytical_f_hat_p(p, m):
    return (2 * m) / (m**2 + p**2)

print("\n--- Example 2.2: Fourier Transform (Direct) ---")
p_val = 1.5
m_val = 0.5
numeric_result_2 = calculate_f_hat_p(p_val, m_val)
analytical_result_2 = analytical_f_hat_p(p_val, m_val)

# The imaginary part should be zero
print(f"f_hat(p={p_val}, m={m_val}) (Numeric):    {numeric_result_2.real:.6f} (Imag: {numeric_result_2.imag:.2f})")
print(f"f_hat(p={p_val}, m={m_val}) (Analytical): {analytical_result_2:.6f}")


Fourier transformations

Inverse Transform of a Gaussian
--- Example 2.1: Fourier Transform (Inverse) ---
f(x=1.0, σ=2.0) (Numeric):    0.176033
f(x=1.0, σ=2.0) (Analytical): 0.176033

Transform of f(x) = exp(-m|x|)

--- Example 2.2: Fourier Transform (Direct) ---
f_hat(p=1.5, m=0.5) (Numeric):    0.400000 (Imag: 0.00)
f_hat(p=1.5, m=0.5) (Analytical): 0.400000


In [3]:
print("\n" + "="*60)
print("Black-Scholes pricing kernel")
print("="*60)

def heat_kernel_component(x, x_prime, tau, sigma):
    """
    Calculates the "heat kernel" component (pure diffusion) of the BS kernel.
    Reference: Equation 532 [cite: 532]
    <x| exp(tau * (sigma^2/2) * d^2/dx^2) |x'>
    """
    term1 = 1 / np.sqrt(2 * np.pi * sigma**2 * tau)
    exponent = - ( (x - x_prime)**2 / (2 * sigma**2 * tau) )
    return term1 * np.exp(exponent)

def black_scholes_pricing_kernel(x, x_prime, tau, r, sigma):
    """
    Calculates the complete Black-Scholes pricing kernel p(x, x', tau).
    Reference: Equation 536 
    
    x = log(S / K_ref) at time tau (today)
    x_prime = log(S' / K_ref) at time 0 (expiration)
    tau = T - t (time to expiration)
    """
    
    # Drift parameter (r - sigma^2/2)
    mu = r - 0.5 * sigma**2
    
    # Denominator of the exponent
    variance_term = 2 * sigma**2 * tau
    
    # Numerator of the exponent
    numerator_exp = - (x - x_prime + mu * tau)**2
    
    # Discount factor and normalization
    factor = np.exp(-r * tau) / np.sqrt(np.pi * variance_term)
    
    return factor * np.exp(numerator_exp / variance_term)

print("\n--- Section 3.4: Black-Scholes Pricing Kernel ---")
# Example parameters
x_today = 0.0     # log(100/100)
x_exp = 0.1       # log(110.5/100)
tau_val = 1.0     # 1 year
r_val = 0.05      # 5%
sigma_val = 0.2   # 20%


# 1. Heat Kernel Component
hk_val = heat_kernel_component(x_today, x_exp, tau_val, sigma_val)
print(f"'Heat Kernel' Component: {hk_val:.6f}")

# 2. Complete Pricing Kernel
pk_val = black_scholes_pricing_kernel(x_today, x_exp, tau_val, r_val, sigma_val)
print(f"Complete Pricing Kernel p(x, x', tau): {pk_val:.6f}")


Black-Scholes pricing kernel

--- Section 3.4: Black-Scholes Pricing Kernel ---
'Heat Kernel' Component: 1.760327
Complete Pricing Kernel p(x, x', tau): 1.784698


In [4]:
print("\n" + "="*60)
print("Option Price (Convolution)")
print("="*60)


# ('black_scholes_pricing_kernel' function defined in Example 2)

def payoff_g(x_prime, K):
    """
    Payoff function g(x) = K(e^x - 1)+
    This is equivalent to max(S' - K, 0) where x' = log(S'/K)
    If x' = log(S'/K), then S' = K * e^x'
    Payoff = max(K * e^x' - K, 0) = K * max(e^x' - 1, 0)
    Reference: Equation 316 
    """
    return K * np.maximum(np.exp(x_prime) - 1, 0)

def integrand_call_price(x_prime, x, tau, r, sigma, K):
    """
    The integrand for the call price: p(x, x', tau) * g(x')
    Reference: Equation 540 / 541 
    """
    kernel = black_scholes_pricing_kernel(x, x_prime, tau, r, sigma)
    payoff = payoff_g(x_prime, K)
    return kernel * payoff

# --- Parameters ---
# We use normalized log-price: x = log(S/K)
S_today = 100.0
K_strike = 105.0
tau_val = 1.0
r_val = 0.05
sigma_val = 0.2

# Current x variable: x = log(S_today / K_strike)
x_current = np.log(S_today / K_strike)

print(f"\n--- Example 2.3 / Section 3.4: Option Price (Convolution) ---")
print(f"Calculating C(x, tau) = integral[ p(x, x', tau) * g(x') dx' ]")
print(f"S={S_today}, K={K_strike}, T={tau_val}, r={r_val}, σ={sigma_val}")
print(f"Current 'x' value (log(S/K)): {x_current:.4f}")

# The integration is over x', which goes from -inf to +inf
# In practice, g(x') is only non-zero when x' > 0 (i.e., S' > K)
# We integrate from 0 to infinity (or a large value)
price, error = quad(
    integrand_call_price, 
    0,                     # Lower limit (where payoff begins)
    np.inf,                # Upper limit
    args=(x_current, tau_val, r_val, sigma_val, K_strike)
)

print(f"\nCall Price (via Numeric Integral): {price:.4f}")
print(f"(Estimated integral error: {error:.2e})")

# Comparison with the standard Black-Scholes formula (from your 1st query)
def black_scholes_call_standard(S, K, T, r, sigma):
    d1 = (np.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    call_price = (S * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2))
    return call_price

standard_price = black_scholes_call_standard(S_today, K_strike, tau_val, r_val, sigma_val)
print(f"Call Price (Standard BS Formula):   {standard_price:.4f}")


Option Price (Convolution)

--- Example 2.3 / Section 3.4: Option Price (Convolution) ---
Calculating C(x, tau) = integral[ p(x, x', tau) * g(x') dx' ]
S=100.0, K=105.0, T=1.0, r=0.05, σ=0.2
Current 'x' value (log(S/K)): -0.0488

Call Price (via Numeric Integral): nan
(Estimated integral error: nan)
Call Price (Standard BS Formula):   8.0214


C:\Users\daves\AppData\Local\Temp\ipykernel_5248\1623404550.py:16: RuntimeWarning: overflow encountered in exp
  return K * np.maximum(np.exp(x_prime) - 1, 0)
C:\Users\daves\AppData\Local\Temp\ipykernel_5248\1623404550.py:25: RuntimeWarning: invalid value encountered in scalar multiply
  return kernel * payoff
C:\Users\daves\AppData\Local\Temp\ipykernel_5248\1623404550.py:46: IntegrationWarning: The occurrence of roundoff error is detected, which prevents 
  the requested tolerance from being achieved.  The error may be 
  underestimated.
  price, error = quad(
